In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import time
import matplotlib.pyplot as plt
import numpy as np

In [2]:
mean_std = pd.read_csv("../data_processing/mean_std.csv", index_col = 0)

In [3]:
checkpoint = torch.load("../test/Brian/nov22_test01/final/checkpoint_final.tar")
demo_df = pd.read_csv("../split_datasets/live_demo.csv")

test_df = pd.read_csv("../split_datasets/test.csv")
test_df = test_df.drop(columns="id")
test_df = test_df.drop(columns="last_scraped")
test_df = test_df.drop(columns="amenities")
demo_df = demo_df.drop(columns="amenities")

FileNotFoundError: [Errno 2] File b'../split_datasets/live_demo.csv' does not exist: b'../split_datasets/live_demo.csv'

In [ ]:
demo_df

In [ ]:
test_df

In [ ]:
mean_std

In [ ]:
def get_error(net, train_loader, threshold):
    train_samples = 0
    train_error = 0
    train_accuracy = [0 for i in threshold]
    for i, data in enumerate(train_loader):
        sample = data["sample"].float()
        label = data["label"].float().view(-1)
        output = net(sample).view(-1)
        error = abs(label-output)*mean_std["std"]["price"]
        for j in range(len(threshold)):
            accurate = error < float(threshold[j])
            train_accuracy[j] += accurate.sum().item()
        train_samples += len(label)
        train_error += error.sum().item()
    avg_train_error = float(train_error)/float(train_samples)
    avg_train_accuracy = [float(a)/float(train_samples) for a in train_accuracy]
    
    return avg_train_error, avg_train_accuracy

In [ ]:
class airbnb_net (nn.Module):
    def __init__(self):
        super(airbnb_net, self).__init__()
        self.layer1 = nn.Linear(41, 41*3)
        self.layer2 = nn.Linear(41*3, 41*3)
        self.layer3 = nn.Linear(41*3, 41*2)
        self.layer4 = nn.Linear(41*2, 41*2)
        self.layer5 = nn.Linear(41*2, 41*1)
        self.layer6 = nn.Linear(41*1, 1)
        
        
        #self.dropout2 = nn.Dropout(p=0.1)
    
    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        #x = self.dropout2(x)
        x = torch.relu(self.layer3(x))
        #x = self.dropout2(x)
        x = torch.relu(self.layer4(x))
        #x = self.dropout2(x)
        x = torch.relu(self.layer5(x))
        return self.layer6(x)

In [ ]:
model = airbnb_net()
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
class airbnb_dataset (Dataset):
    def __init__(self, df, purpose):
        self.price = torch.from_numpy(df["price"].to_numpy())
        self.data = torch.from_numpy(df.drop(columns="price").to_numpy())        
        self.purpose = purpose
        
    def __len__(self):
        return len(self.price)
    
    def __getitem__(self, index):
        # Returns specific sample as a dict
        if torch.is_tensor(index):
            index = index.tolist()
        sample = self.data[index]
        label = self.price[index]
        
        return {"sample": sample, "label": label}
           

test_ds = airbnb_dataset(test_df, "test")
demo_ds = airbnb_dataset(demo_df, "demo")

demo_loader = DataLoader(demo_ds, batch_size=1000, shuffle=False, num_workers=0)
test_loader = DataLoader(test_ds, batch_size=1000, shuffle=False, num_workers=0)

In [ ]:
get_error(model, test_loader, [10, 20, 30])

In [ ]:
get_error(model, demo_loader, [10, 20, 30])

In [ ]:
test_ds = airbnb_dataset(test_df[:10], "test")
demo_ds = airbnb_dataset(demo_df, "demo")

demo_loader = DataLoader(demo_ds, batch_size=1000, shuffle=False, num_workers=0)
test_loader = DataLoader(test_ds, batch_size=1000, shuffle=False, num_workers=0)

In [4]:
for i, data in enumerate(test_loader):
        sample = data["sample"].float()
        label = data["label"].float().view(-1)
        output = model(sample).view(-1)
        error = abs(label-output)*mean_std["std"]["price"]
output_list = output.tolist()
label_list = label.tolist()
for i in range(0, len(output_list)):
    output_list[i] = (output_list[i]*mean_std["std"]["price"])+mean_std["mean"]["price"]
    label_list[i] = (label_list[i]*mean_std["std"]["price"])+mean_std["mean"]["price"]

print(label_list)
#print(output_list)
print(error)
label_list2 = []
output_list2 = []
    

plt.plot(label_list, label="Real Price")
plt.plot(output_list, label="Predicted Price")

plt.xlabel("House")
plt.ylabel("Price ($)")
plt.title("Price")
plt.legend(loc='best')

NameError: name 'test_loader' is not defined